In [1]:
#!pip install konlpy fasttext

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import urllib.request
from collections import Counter

%matplotlib inline
import re
from konlpy.tag import Twitter
import csv

from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# df = pd.read_csv("/content/drive/MyDrive/DATA/review.csv", encoding='cp949')
df = pd.read_csv("/content/drive/MyDrive/DATA/baemin_review.csv")
df.drop(
    columns=['닉네임', '날짜', '번호', '배달평가(상,중,하)', '주문메뉴1', '주문메뉴2', '주문메뉴3', '주문메뉴4', '사진1', '사진2']
    , errors='ignore'
    , inplace=True
)

df

,별점(1~5),리뷰
0,5.0,시켜먹고 그 다음날 또 시키게 만드는 마력의 디에스티... 이틀연속 시켜먹었습니다....
1,5.0,저의 최애 크로플가게입니다. 오늘도 서비스가 대단히 대단합니다^^ 전 그릭요거트만 ...
2,5.0,진짜 맛있어요..!! 딸기도 완전 많구 크림도 안느끼해여 코코아파우더를 뿌려주신건지...
3,5.0,서비스까지 넘무 감사합니다 ㅠ 믿고..먹는..디에스티.. 크로플도 최고입니다ㅜ 번창...
4,5.0,크로플 맛있어요! 애플시나몬에 사과가 찐이에요! 샤인청포도도 찐 청포도라 맛있어요 ...
...,...,...
1060,5.0,라떼는 할아버지 할머니가 드실거라고 글 남겼는데 우유 락토프리로 할건지 일반우유로 ...
1061,5.0,사진은 못찍었어요ㅜㅜ 근데 맛있게마셨어요~~ 전 밀크티 차향이 좀 나는걸 좋아하는데...
1062,5.0,드라마 보다가 오키나와를 다시 가고 싶어져서 아무 생각없이 오키나와를 검색했는데 여...
1063,5.0,오 디에스티 배달도 되네요~맛있어요 ㅎㅎ


In [5]:
df = df[(df['리뷰'].notna())]

df['target'] = df['별점(1~5)'].apply(lambda x: x/5)

df['target'].value_counts()

# # # label = 평점이 5인 리뷰에는 레이블 1을, 평점이 1, 2인 리뷰에는 레이블 0

# # df['label'] = np.select([df.별점 >= 5], [1], default=0)
# # df[:5]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1.0    919
0.8     22
0.6     10
0.2      5
0.4      4
Name: target, dtype: int64

In [6]:
# # class_weights = {
# #     0: 0
# #     , 1: 0
# # }

# get_class_weights = df['target'].value_counts().to_dict()
# v_max = max(get_class_weights.values())

# for k,v in get_class_weights.items():
#     if k < 1:
#         k = 0
#     else:
#         k = 1

#     class_weights[k] += v/v_max

# class_weights

In [7]:
# 클래스 컬럼 추가
df['class'] = df['별점(1~5)'].apply(lambda x : x-1)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,별점(1~5),리뷰,target,class
0,5.0,시켜먹고 그 다음날 또 시키게 만드는 마력의 디에스티... 이틀연속 시켜먹었습니다....,1.0,4.0
1,5.0,저의 최애 크로플가게입니다. 오늘도 서비스가 대단히 대단합니다^^ 전 그릭요거트만 ...,1.0,4.0
2,5.0,진짜 맛있어요..!! 딸기도 완전 많구 크림도 안느끼해여 코코아파우더를 뿌려주신건지...,1.0,4.0
3,5.0,서비스까지 넘무 감사합니다 ㅠ 믿고..먹는..디에스티.. 크로플도 최고입니다ㅜ 번창...,1.0,4.0
4,5.0,크로플 맛있어요! 애플시나몬에 사과가 찐이에요! 샤인청포도도 찐 청포도라 맛있어요 ...,1.0,4.0
...,...,...,...,...
1059,5.0,여기 흑당밀크티는 맛있어서 자주 먹으러 오는데 신메뉴 나왔다길래 신메뉴도 주문해 봤...,1.0,4.0
1060,5.0,라떼는 할아버지 할머니가 드실거라고 글 남겼는데 우유 락토프리로 할건지 일반우유로 ...,1.0,4.0
1061,5.0,사진은 못찍었어요ㅜㅜ 근데 맛있게마셨어요~~ 전 밀크티 차향이 좀 나는걸 좋아하는데...,1.0,4.0
1062,5.0,드라마 보다가 오키나와를 다시 가고 싶어져서 아무 생각없이 오키나와를 검색했는데 여...,1.0,4.0


In [8]:
class_weights = df['class'].value_counts().to_dict()
v_max = len(df.index)
class_weights = {int(k): 1-v/v_max for k,v in class_weights.items()}
# class_weights = [v/v_max for k,v in class_weights.items()]
class_weights

{0: 0.9947916666666666,
 1: 0.9958333333333333,
 2: 0.9895833333333334,
 3: 0.9770833333333333,
 4: 0.04270833333333335}

In [9]:
# from sklearn.utils import class_weight 

# class_weight.compute_class_weight(
#     'balanced'
#     , np.unique(df['별점(1~5)'])
#     , df['별점(1~5)']
# )
# # class_weights = 
# # class_weights = dict(enumerate(class_weights))

In [10]:
# 형태소 변환
t = Twitter()

def conv_pos_text(trg_txt):
    return ' '.join([w for w, _ in t.pos(trg_txt, norm=True)])

df['pos_text'] = df['리뷰'].apply(lambda x: conv_pos_text(x))
df

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,별점(1~5),리뷰,target,class,pos_text
0,5.0,시켜먹고 그 다음날 또 시키게 만드는 마력의 디에스티... 이틀연속 시켜먹었습니다....,1.0,4.0,시켜 먹고 그 다음 날 또 시키게 만드는 마력 의 디 에스티 ... 이틀 연속 시켜...
1,5.0,저의 최애 크로플가게입니다. 오늘도 서비스가 대단히 대단합니다^^ 전 그릭요거트만 ...,1.0,4.0,저 의 최애 크로 플 가게 입니다 . 오늘 도 서비스 가 대단히 대단합니다 ^^ 전...
2,5.0,진짜 맛있어요..!! 딸기도 완전 많구 크림도 안느끼해여 코코아파우더를 뿌려주신건지...,1.0,4.0,진짜 맛있어요 ..!! 딸기 도 완전 많구 크림 도 안 느끼해여 코코아파우더 를 뿌...
3,5.0,서비스까지 넘무 감사합니다 ㅠ 믿고..먹는..디에스티.. 크로플도 최고입니다ㅜ 번창...,1.0,4.0,서비스 까지 넘 무 감사합니다 ㅠ 믿고 .. 먹는 .. 디 에스티 .. 크로 플 도...
4,5.0,크로플 맛있어요! 애플시나몬에 사과가 찐이에요! 샤인청포도도 찐 청포도라 맛있어요 ...,1.0,4.0,크로 플 맛있어요 ! 애플 시나몬 에 사과 가 찐이에요 ! 샤인 청포도 도 찐 청포...
...,...,...,...,...,...
1059,5.0,여기 흑당밀크티는 맛있어서 자주 먹으러 오는데 신메뉴 나왔다길래 신메뉴도 주문해 봤...,1.0,4.0,여기 흑 당 밀크 티 는 맛있어서 자주 먹으러 오는데 신 메뉴 나왔다길래 신 메뉴 ...
1060,5.0,라떼는 할아버지 할머니가 드실거라고 글 남겼는데 우유 락토프리로 할건지 일반우유로 ...,1.0,4.0,라떼 는 할아버지 할머니 가 드실거라고 글 남겼는데 우유 락토프리 로 할 건지 일 ...
1061,5.0,사진은 못찍었어요ㅜㅜ 근데 맛있게마셨어요~~ 전 밀크티 차향이 좀 나는걸 좋아하는데...,1.0,4.0,사진 은 못 찍었어요 ㅜㅜ 근데 맛있게 마셨어요 ~~ 전 밀크 티 차향 이 좀 나는...
1062,5.0,드라마 보다가 오키나와를 다시 가고 싶어져서 아무 생각없이 오키나와를 검색했는데 여...,1.0,4.0,드라마 보다가 오키나와 를 다시 가고 싶어져서 아무 생각 없이 오키나와 를 검색 했...


In [11]:
train_data_path = '/content/drive/MyDrive/DATA/text_only.txt'
df[['target', 'pos_text']].to_csv(
    train_data_path, 
    index = False, 
    sep = ' ',
    header = None, 
    quoting = csv.QUOTE_NONE, 
    quotechar = "", 
    escapechar = " "
)

In [12]:
import fasttext

ft_model = fasttext.train_supervised(
    input=train_data_path
    , lr=0.5
    , epoch=50, wordNgrams=4, bucket=200000, dim=100, loss='ova'
    , verbose=1
)

In [13]:
ft_model.save_model("/content/drive/MyDrive/MODEL/vectors.bin")

In [14]:
df['text2vec'] = df['pos_text'].apply(lambda x: ft_model.get_sentence_vector(x))
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,별점(1~5),리뷰,target,class,pos_text,text2vec
0,5.0,시켜먹고 그 다음날 또 시키게 만드는 마력의 디에스티... 이틀연속 시켜먹었습니다....,1.0,4.0,시켜 먹고 그 다음 날 또 시키게 만드는 마력 의 디 에스티 ... 이틀 연속 시켜...,"[-0.00023126372, 0.0001614271, 0.00018011549, ..."
1,5.0,저의 최애 크로플가게입니다. 오늘도 서비스가 대단히 대단합니다^^ 전 그릭요거트만 ...,1.0,4.0,저 의 최애 크로 플 가게 입니다 . 오늘 도 서비스 가 대단히 대단합니다 ^^ 전...,"[0.0002444432, 0.0003062245, -0.00020172272, -..."
2,5.0,진짜 맛있어요..!! 딸기도 완전 많구 크림도 안느끼해여 코코아파우더를 뿌려주신건지...,1.0,4.0,진짜 맛있어요 ..!! 딸기 도 완전 많구 크림 도 안 느끼해여 코코아파우더 를 뿌...,"[0.0002192988, 0.0003585422, 0.00025331983, -0..."
3,5.0,서비스까지 넘무 감사합니다 ㅠ 믿고..먹는..디에스티.. 크로플도 최고입니다ㅜ 번창...,1.0,4.0,서비스 까지 넘 무 감사합니다 ㅠ 믿고 .. 먹는 .. 디 에스티 .. 크로 플 도...,"[-2.6501488e-05, -0.00031201515, -0.0001158372..."
4,5.0,크로플 맛있어요! 애플시나몬에 사과가 찐이에요! 샤인청포도도 찐 청포도라 맛있어요 ...,1.0,4.0,크로 플 맛있어요 ! 애플 시나몬 에 사과 가 찐이에요 ! 샤인 청포도 도 찐 청포...,"[0.00011734804, -0.00010148594, -0.0005383307,..."
...,...,...,...,...,...,...
1059,5.0,여기 흑당밀크티는 맛있어서 자주 먹으러 오는데 신메뉴 나왔다길래 신메뉴도 주문해 봤...,1.0,4.0,여기 흑 당 밀크 티 는 맛있어서 자주 먹으러 오는데 신 메뉴 나왔다길래 신 메뉴 ...,"[-6.850717e-07, -9.3963956e-07, -0.00037235313..."
1060,5.0,라떼는 할아버지 할머니가 드실거라고 글 남겼는데 우유 락토프리로 할건지 일반우유로 ...,1.0,4.0,라떼 는 할아버지 할머니 가 드실거라고 글 남겼는데 우유 락토프리 로 할 건지 일 ...,"[0.00015900518, 0.00013407008, -0.00027049723,..."
1061,5.0,사진은 못찍었어요ㅜㅜ 근데 맛있게마셨어요~~ 전 밀크티 차향이 좀 나는걸 좋아하는데...,1.0,4.0,사진 은 못 찍었어요 ㅜㅜ 근데 맛있게 마셨어요 ~~ 전 밀크 티 차향 이 좀 나는...,"[0.0006388988, 0.00013842409, 0.00034687802, 9..."
1062,5.0,드라마 보다가 오키나와를 다시 가고 싶어져서 아무 생각없이 오키나와를 검색했는데 여...,1.0,4.0,드라마 보다가 오키나와 를 다시 가고 싶어져서 아무 생각 없이 오키나와 를 검색 했...,"[0.00012428508, 0.00015264553, -1.2462766e-05,..."


In [15]:
X = np.vstack(df['text2vec'])
# multi-class
y = df['class']
# # binary
# y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, shuffle=True, stratify=y)

In [16]:
# 패키지 준비
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import *


In [17]:
# 모델 설계
model = Sequential()
model.add(Embedding(len(ft_model.words)+1, 100))
# model.add(SpatialDropout1D(0.2))

# model.add(LSTM(128))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))

# model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))

# multi-class
model.add(Dense(units=5, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

# binary
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['mse'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         505700    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        84480     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 635,881
Trainable params: 635,881
Non-trai

In [18]:
# 모델 검증
trg_metrics = 'loss' # auc, acc, loss
mode = 'min'
# binary, multi-class
es = EarlyStopping(monitor=f'{trg_metrics}', mode=mode, verbose=1, patience=10)
mc = ModelCheckpoint(
    '/content/drive/MyDrive/MODEL/best_model.h5'
    , monitor=f'val_{trg_metrics}'
    , mode=mode
    , verbose=1
    , save_best_only=True
)

In [19]:
# 모델 훈련
history = model.fit(
    X_train
    , y_train
    , epochs=20
    , callbacks=[es, mc]
    , validation_data=(X_test, y_test)
    , batch_size=None
    , class_weight=class_weights
)

Epoch 1/20
26/26 [==============================] - ETA: 0s - loss: 0.1224 - acc: 0.9203
Epoch 1: val_loss improved from inf to 0.36310, saving model to /content/drive/MyDrive/MODEL/best_model.h5
26/26 [==============================] - 21s 307ms/step - loss: 0.1224 - acc: 0.9203 - val_loss: 0.3631 - val_acc: 0.9583
Epoch 2/20
26/26 [==============================] - ETA: 0s - loss: 0.1097 - acc: 0.9571
Epoch 2: val_loss did not improve from 0.36310
26/26 [==============================] - 6s 224ms/step - loss: 0.1097 - acc: 0.9571 - val_loss: 0.9067 - val_acc: 0.9583
Epoch 3/20
26/26 [==============================] - ETA: 0s - loss: 0.1082 - acc: 0.9571
Epoch 3: val_loss did not improve from 0.36310
26/26 [==============================] - 5s 207ms/step - loss: 0.1082 - acc: 0.9571 - val_loss: 0.8219 - val_acc: 0.9583
Epoch 4/20
26/26 [==============================] - ETA: 0s - loss: 0.1084 - acc: 0.9571
Epoch 4: val_loss did not improve from 0.36310
26/26 [=========================

In [20]:
# 긍정(별5개) 데이터
#sample_data = df.iloc[500]

# # 부정(별1개) 데이터
sample_data = df[(df['별점(1~5)'] == 1)].iloc[-1]

print(f"텍스트 : {sample_data['pos_text']}")
print(f"벡터화 : {sample_data['text2vec']}")
print(f"정답 : {sample_data['class']}")

텍스트 : 웬만하면 리뷰 안 쓰는데요 늦은 건 그렇다치고 이 걸 상품 이라고 파는 게 말 이 되나 싶어서 올립니다 다 타서 먹지도 않고 그냥 버렸습니다 퇴근 시간 겹 쳐서 바쁘 신건 알겠는데 그럴거면 그냥 주 문 취소 를 시켜주세요
벡터화 : [-7.1363524e-05 -1.9326423e-04 -5.7919585e-04 -1.7463963e-04
 -8.0940008e-05 -1.2970688e-04 -1.6095760e-04 -4.8994385e-05
  5.5490719e-04  4.9840717e-04 -5.0644365e-05  1.2960217e-04
  2.7191936e-04  1.2698754e-04 -5.6236511e-04 -1.1333798e-04
  4.4558168e-04 -2.7650068e-04  7.9919200e-04 -2.2673086e-04
  9.5962670e-05  2.0153332e-06  4.3995451e-04 -2.7731346e-04
 -3.7004179e-04  3.5477009e-05  7.7863937e-05 -3.5753785e-04
  2.0689786e-04 -2.6415382e-04 -1.5066503e-04 -4.4552929e-04
 -5.8613095e-04 -4.1927480e-05  1.3774008e-04  2.3708642e-04
  4.7514757e-05 -5.1760598e-04 -8.9840956e-05  2.2802666e-04
 -3.8338720e-04 -2.1093493e-04  1.4791066e-04  1.6584381e-04
 -1.3288758e-04 -2.3108977e-04 -4.4556637e-05  1.7933513e-04
 -2.6802043e-05  4.3428756e-04 -2.4411514e-04 -1.8304947e-04
 -6.4707198e-04  6.6628098e-04 -3.4564984e-04  2.0571856e-04
  4.336129

In [21]:
pred_y = model.predict(np.expand_dims(sample_data['text2vec'], axis=0))
print(f"예측값 : {pred_y[0]}")

예측값 : [0.06077939 0.04400067 0.1080139  0.30215317 0.4850529 ]


In [22]:
# pred_y[np.arange(len(pred_y))[:, None], np.argsort(pred_y)[:, -2:]][:, ::-1]

idx = np.argmax(pred_y)
pred_val = pred_y[0][idx]

print(f"{idx} {pred_val}")

4 0.4850529134273529


In [23]:
df['class_ft'] = df['별점(1~5)'].apply(lambda x : f"__label__{int(x)}")
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,별점(1~5),리뷰,target,class,pos_text,text2vec,class_ft
0,5.0,시켜먹고 그 다음날 또 시키게 만드는 마력의 디에스티... 이틀연속 시켜먹었습니다....,1.0,4.0,시켜 먹고 그 다음 날 또 시키게 만드는 마력 의 디 에스티 ... 이틀 연속 시켜...,"[-0.00023126372, 0.0001614271, 0.00018011549, ...",__label__5
1,5.0,저의 최애 크로플가게입니다. 오늘도 서비스가 대단히 대단합니다^^ 전 그릭요거트만 ...,1.0,4.0,저 의 최애 크로 플 가게 입니다 . 오늘 도 서비스 가 대단히 대단합니다 ^^ 전...,"[0.0002444432, 0.0003062245, -0.00020172272, -...",__label__5
2,5.0,진짜 맛있어요..!! 딸기도 완전 많구 크림도 안느끼해여 코코아파우더를 뿌려주신건지...,1.0,4.0,진짜 맛있어요 ..!! 딸기 도 완전 많구 크림 도 안 느끼해여 코코아파우더 를 뿌...,"[0.0002192988, 0.0003585422, 0.00025331983, -0...",__label__5
3,5.0,서비스까지 넘무 감사합니다 ㅠ 믿고..먹는..디에스티.. 크로플도 최고입니다ㅜ 번창...,1.0,4.0,서비스 까지 넘 무 감사합니다 ㅠ 믿고 .. 먹는 .. 디 에스티 .. 크로 플 도...,"[-2.6501488e-05, -0.00031201515, -0.0001158372...",__label__5
4,5.0,크로플 맛있어요! 애플시나몬에 사과가 찐이에요! 샤인청포도도 찐 청포도라 맛있어요 ...,1.0,4.0,크로 플 맛있어요 ! 애플 시나몬 에 사과 가 찐이에요 ! 샤인 청포도 도 찐 청포...,"[0.00011734804, -0.00010148594, -0.0005383307,...",__label__5
...,...,...,...,...,...,...,...
1059,5.0,여기 흑당밀크티는 맛있어서 자주 먹으러 오는데 신메뉴 나왔다길래 신메뉴도 주문해 봤...,1.0,4.0,여기 흑 당 밀크 티 는 맛있어서 자주 먹으러 오는데 신 메뉴 나왔다길래 신 메뉴 ...,"[-6.850717e-07, -9.3963956e-07, -0.00037235313...",__label__5
1060,5.0,라떼는 할아버지 할머니가 드실거라고 글 남겼는데 우유 락토프리로 할건지 일반우유로 ...,1.0,4.0,라떼 는 할아버지 할머니 가 드실거라고 글 남겼는데 우유 락토프리 로 할 건지 일 ...,"[0.00015900518, 0.00013407008, -0.00027049723,...",__label__5
1061,5.0,사진은 못찍었어요ㅜㅜ 근데 맛있게마셨어요~~ 전 밀크티 차향이 좀 나는걸 좋아하는데...,1.0,4.0,사진 은 못 찍었어요 ㅜㅜ 근데 맛있게 마셨어요 ~~ 전 밀크 티 차향 이 좀 나는...,"[0.0006388988, 0.00013842409, 0.00034687802, 9...",__label__5
1062,5.0,드라마 보다가 오키나와를 다시 가고 싶어져서 아무 생각없이 오키나와를 검색했는데 여...,1.0,4.0,드라마 보다가 오키나와 를 다시 가고 싶어져서 아무 생각 없이 오키나와 를 검색 했...,"[0.00012428508, 0.00015264553, -1.2462766e-05,...",__label__5


In [24]:
train_data_path = '/content/drive/MyDrive/DATA/text_only2.txt'
df[['class_ft', 'pos_text']].to_csv(
    train_data_path, 
    index = False, 
    sep = ' ',
    header = None, 
    quoting = csv.QUOTE_NONE, 
    quotechar = "", 
    escapechar = " "
)

In [25]:
import fasttext

ft_model = fasttext.train_supervised(
    input=train_data_path
    , lr=0.5
    , epoch=100
    , wordNgrams=4
    , bucket=200000
    , dim=100
    , loss='ova'
)

In [26]:
ft_model.predict(sample_data['pos_text'])

(('__label__1',), array([0.49219814]))

In [27]:
ft_model.predict(conv_pos_text("진짜 맛있어요 대박 나세요"))

(('__label__5',), array([1.00001001]))